Creating a visualization that goes frame by frame through an NFL play

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [14]:
tracking_week_1 = pd.read_csv('../data/tracking_week_1.csv')
plays = pd.read_csv('../data/plays.csv')

In [15]:
df = pd.merge(tracking_week_1, plays, on=['gameId', 'playId'])
df.head()

,gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,...,yardsGained,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPointsAdded,isDropback,pff_runConceptPrimary,pff_runConceptSecondary,pff_runPassOption,pff_passCoverage,pff_manZone
0,2022091200,64,35459.0,Kareem Jackson,1,BEFORE_SNAP,2022-09-13 00:16:03.5,22.0,DEN,right,...,0,0.004575,-0.004575,-0.362858,False,OUTSIDE ZONE,LEAD;PITCH,0,Cover-3,Zone
1,2022091200,64,35459.0,Kareem Jackson,2,BEFORE_SNAP,2022-09-13 00:16:03.6,22.0,DEN,right,...,0,0.004575,-0.004575,-0.362858,False,OUTSIDE ZONE,LEAD;PITCH,0,Cover-3,Zone
2,2022091200,64,35459.0,Kareem Jackson,3,BEFORE_SNAP,2022-09-13 00:16:03.7,22.0,DEN,right,...,0,0.004575,-0.004575,-0.362858,False,OUTSIDE ZONE,LEAD;PITCH,0,Cover-3,Zone
3,2022091200,64,35459.0,Kareem Jackson,4,BEFORE_SNAP,2022-09-13 00:16:03.8,22.0,DEN,right,...,0,0.004575,-0.004575,-0.362858,False,OUTSIDE ZONE,LEAD;PITCH,0,Cover-3,Zone
4,2022091200,64,35459.0,Kareem Jackson,5,BEFORE_SNAP,2022-09-13 00:16:03.9,22.0,DEN,right,...,0,0.004575,-0.004575,-0.362858,False,OUTSIDE ZONE,LEAD;PITCH,0,Cover-3,Zone


In [16]:
df.columns

Index(['gameId', 'playId', 'nflId', 'displayName', 'frameId', 'frameType',
       'time', 'jerseyNumber', 'club', 'playDirection', 'x', 'y', 's', 'a',
       'dis', 'o', 'dir', 'event', 'playDescription', 'quarter', 'down',
       'yardsToGo', 'possessionTeam', 'defensiveTeam', 'yardlineSide',
       'yardlineNumber', 'gameClock', 'preSnapHomeScore',
       'preSnapVisitorScore', 'playNullifiedByPenalty',
       'absoluteYardlineNumber', 'preSnapHomeTeamWinProbability',
       'preSnapVisitorTeamWinProbability', 'expectedPoints',
       'offenseFormation', 'receiverAlignment', 'playClockAtSnap',
       'passResult', 'passLength', 'targetX', 'targetY', 'playAction',
       'dropbackType', 'dropbackDistance', 'passLocationType', 'timeToThrow',
       'timeInTackleBox', 'timeToSack', 'passTippedAtLine',
       'unblockedPressure', 'qbSpike', 'qbKneel', 'qbSneak',
       'rushLocationType', 'penaltyYards', 'prePenaltyYardsGained',
       'yardsGained', 'homeTeamWinProbabilityAdded',
      

In [ ]:
# input gameid and playid here

df = df[df['gameId'] == 2022091100]
df = df[df['playId'] == 546]
df.head()

,gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,...,yardsGained,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPointsAdded,isDropback,pff_runConceptPrimary,pff_runConceptSecondary,pff_runPassOption,pff_passCoverage,pff_manZone
6285762,2022091100,546,38592.0,Casey Hayward,1,BEFORE_SNAP,2022-09-11 17:28:53,29.0,ATL,left,...,57,-0.103411,0.103411,3.198946,False,POWER,LEAD;QB RUNS,0,Goal Line,Other
6285763,2022091100,546,38592.0,Casey Hayward,2,BEFORE_SNAP,2022-09-11 17:28:53.1,29.0,ATL,left,...,57,-0.103411,0.103411,3.198946,False,POWER,LEAD;QB RUNS,0,Goal Line,Other
6285764,2022091100,546,38592.0,Casey Hayward,3,BEFORE_SNAP,2022-09-11 17:28:53.2,29.0,ATL,left,...,57,-0.103411,0.103411,3.198946,False,POWER,LEAD;QB RUNS,0,Goal Line,Other
6285765,2022091100,546,38592.0,Casey Hayward,4,BEFORE_SNAP,2022-09-11 17:28:53.3,29.0,ATL,left,...,57,-0.103411,0.103411,3.198946,False,POWER,LEAD;QB RUNS,0,Goal Line,Other
6285766,2022091100,546,38592.0,Casey Hayward,5,BEFORE_SNAP,2022-09-11 17:28:53.4,29.0,ATL,left,...,57,-0.103411,0.103411,3.198946,False,POWER,LEAD;QB RUNS,0,Goal Line,Other


In [18]:
df = df.sort_values(by='frameId')
df.head()

,gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,...,yardsGained,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPointsAdded,isDropback,pff_runConceptPrimary,pff_runConceptSecondary,pff_runPassOption,pff_passCoverage,pff_manZone
6285762,2022091100,546,38592.0,Casey Hayward,1,BEFORE_SNAP,2022-09-11 17:28:53,29.0,ATL,left,...,57,-0.103411,0.103411,3.198946,False,POWER,LEAD;QB RUNS,0,Goal Line,Other
6287994,2022091100,546,48520.0,Anthony Rush,1,BEFORE_SNAP,2022-09-11 17:28:53,94.0,ATL,left,...,57,-0.103411,0.103411,3.198946,False,POWER,LEAD;QB RUNS,0,Goal Line,Other
6287808,2022091100,546,47831.0,Erik McCoy,1,BEFORE_SNAP,2022-09-11 17:28:53,78.0,NO,left,...,57,-0.103411,0.103411,3.198946,False,POWER,LEAD;QB RUNS,0,Goal Line,Other
6286320,2022091100,546,42344.0,Jameis Winston,1,BEFORE_SNAP,2022-09-11 17:28:53,2.0,NO,left,...,57,-0.103411,0.103411,3.198946,False,POWER,LEAD;QB RUNS,0,Goal Line,Other
6289482,2022091100,546,53932.0,Adam Prentice,1,BEFORE_SNAP,2022-09-11 17:28:53,46.0,NO,left,...,57,-0.103411,0.103411,3.198946,False,POWER,LEAD;QB RUNS,0,Goal Line,Other


In [19]:
df.columns

Index(['gameId', 'playId', 'nflId', 'displayName', 'frameId', 'frameType',
       'time', 'jerseyNumber', 'club', 'playDirection', 'x', 'y', 's', 'a',
       'dis', 'o', 'dir', 'event', 'playDescription', 'quarter', 'down',
       'yardsToGo', 'possessionTeam', 'defensiveTeam', 'yardlineSide',
       'yardlineNumber', 'gameClock', 'preSnapHomeScore',
       'preSnapVisitorScore', 'playNullifiedByPenalty',
       'absoluteYardlineNumber', 'preSnapHomeTeamWinProbability',
       'preSnapVisitorTeamWinProbability', 'expectedPoints',
       'offenseFormation', 'receiverAlignment', 'playClockAtSnap',
       'passResult', 'passLength', 'targetX', 'targetY', 'playAction',
       'dropbackType', 'dropbackDistance', 'passLocationType', 'timeToThrow',
       'timeInTackleBox', 'timeToSack', 'passTippedAtLine',
       'unblockedPressure', 'qbSpike', 'qbKneel', 'qbSneak',
       'rushLocationType', 'penaltyYards', 'prePenaltyYardsGained',
       'yardsGained', 'homeTeamWinProbabilityAdded',
      

In [20]:
df = df[['displayName', 'frameId', 'jerseyNumber', 'club', 'x', 'y', 'possessionTeam', 'defensiveTeam', 'gameClock', 'preSnapHomeScore',
       'preSnapVisitorScore', ]]
df.head()

,displayName,frameId,jerseyNumber,club,x,y,possessionTeam,defensiveTeam,gameClock,preSnapHomeScore,preSnapVisitorScore
6285762,Casey Hayward,1,29.0,ATL,76.17,23.35,NO,ATL,06:33,3,0
6287994,Anthony Rush,1,94.0,ATL,76.10,29.99,NO,ATL,06:33,3,0
6287808,Erik McCoy,1,78.0,NO,80.86,30.04,NO,ATL,06:33,3,0
6286320,Jameis Winston,1,2.0,NO,81.58,31.15,NO,ATL,06:33,3,0
6289482,Adam Prentice,1,46.0,NO,82.43,31.57,NO,ATL,06:33,3,0


In [21]:
vis = df[['frameId','club','x','y','possessionTeam']]

In [27]:
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import pandas as pd

class NFLPlayVisualization:
    def __init__(self, dataframe):
        self.df = dataframe
        
        # Field dimensions
        self.FIELD_LENGTH = 120  # yards
        self.FIELD_WIDTH = 53.3  # yards
        
        # Setup the figure and axis
        plt.close('all')  # Close any existing plots
        self.fig, self.ax = plt.subplots(figsize=(12, 6))
        
        # Prepare the color mapping
        self.color_map = {
            'ARI':"red",
            'ATL':"red",
            'BAL':"purple",
            'BUF':"blue",
            'CAR':"blue",
            'CHI':"orange",
            'CIN':"orange",
            'CLE':"orange",
            'DAL':"blue",
            'DEN':"orange",
            'DET':"blue",
            'GB' :"green",
            'HOU':"red",
            'IND':"blue",
            'JAX':"brown",
            'KC' :"red",
            'LA' :"blue",
            'LAC':"yellow",
            'LV' :"black",
            'MIA':"teal",
            'MIN':"purple",
            'NE' :"silver",
            'NO' :"gold",
            'NYG':"blue",
            'NYJ':"green",
            'PHI':"green",
            'PIT':"yellow",
            'SEA':"blue",
            'SF' :"red",
            'TB' :"red",
            'TEN':"blue",
            'WAS':"red",
            'football':"brown"
        }
        
        # Setup the field
        self._setup_field()
        
    def _setup_field(self):
        """Setup the football field background"""
        self.ax.clear()
        self.ax.set_xlim(0, self.FIELD_LENGTH)
        self.ax.set_ylim(0, self.FIELD_WIDTH)
        
        # Field color
        self.ax.set_facecolor('green')
        
        # Field lines
        self.ax.axhline(0, color='white', linewidth=2)
        self.ax.axhline(self.FIELD_WIDTH, color='white', linewidth=2)
        
        # Yard lines
        for x in range(0, self.FIELD_LENGTH+1, 10):
            self.ax.axvline(x, color='white', linestyle='--', alpha=0.5)
        
        self.ax.set_title('NFL Play Visualization')
        self.ax.set_xlabel('Yards')
        self.ax.set_ylabel(' ')
        
    def animate(self, frame):
        """
        Animation function for matplotlib
        """
        # Clear and reset field
        self._setup_field()
        
        # Get data for current frame
        frame_data = self.df[self.df['frameId'] == self.unique_frames[frame]]
        
        # Plot players
        for _, player in frame_data.iterrows():
            color = self.color_map.get(player['club'], 'red')
            
            self.ax.scatter(
                player['x'], 
                player['y'], 
                color=color, 
                s=200,  # Size of the dot
                alpha=0.7,
                edgecolors='black'
            )
            
            # Add jersey number
            self.ax.text(
                player['x'], 
                player['y'], 
                str(int(player['jerseyNumber'])) if not pd.isna(player['jerseyNumber']) else '', 
                color='white', 
                ha='center', 
                va='center',
                fontweight='bold'
            )
        
        # Update title with current frame and game clock
        game_clock = frame_data['gameClock'].iloc[0] if not frame_data.empty else ''
        self.ax.set_title(f'NFL Play Visualization - Frame {frame+1} | Clock: {game_clock}')
        
        return self.ax
    
    def visualize(self, save_path='../gif/nfl_play.gif', interval=150):
        """
        Create the animation
        """
        # Get unique frames
        self.unique_frames = sorted(self.df['frameId'].unique())
        
        # Create animation
        anim = animation.FuncAnimation(
            self.fig, 
            self.animate, 
            frames=len(self.unique_frames),
            interval=interval,
            repeat=False
        )
        
        # Save the animation
        anim.save(save_path, writer='pillow', fps=10)
        
        # Close the plot to free up memory
        plt.close(self.fig)
        
        return anim

In [28]:
# Example Usage:
viz = NFLPlayVisualization(df)
viz.visualize()